In [236]:
import random
import csv
import numpy as np
import pandas as pd
import Loader
import tensorflow as tf
import Transformer
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError
#use for dark mode
plt.rcParams["font.family"] = "serif"
plt.style.use('dark_background')

In [242]:
def runBasicNN(Xtrain,Ytrain, Xval, Yval):
    l1Reg = keras.regularizers.L1(.001)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False
    )
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(len(labelsTrain), activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(200, activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(300, activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(200, activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(100, activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(50, activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(10, activation='relu', kernel_regularizer = l1Reg),
        tf.keras.layers.Dense(2)
    ])

    model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

    #Xtrain,_,valData = Transformer.normalizeData(Xtrain, compoundDataTest, compoundDataValidate, newMean=0, newStd=1)

    history = model.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=30, batch_size=4, verbose = 0)
#     predictionsTrain = np.argmax(model.predict(Xtrain), axis=1)
#     predictionsVal = np.argmax(model.predict(valData), axis=1)
    #acc = np.mean(predictionsTrain == Ytrain)
    predictionsTrain = history.history["val_accuracy"][-1]
    predictionsVal = history.history["accuracy"][-1]
        #print(acc)
    return [predictionsTrain, predictionsVal]

def get_data():
    compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
    compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
    compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

    labelsMean, trainMean = Transformer.useAverageFD(labelsTrain, compoundDataTrain)
    _, testMean = Transformer.useAverageFD(labelsTest, compoundDataTest)
    _, valMean = Transformer.useAverageFD(labelsValidate, compoundDataValidate)

    labelsMax, trainMax = Transformer.useMaxFD(labelsTrain, compoundDataTrain)
    _, testMax = Transformer.useMaxFD(labelsTest, compoundDataTest)
    _, valMax = Transformer.useMaxFD(labelsValidate, compoundDataValidate)

    #after transformations are done assign data
    dataLabels = labelsMax
    trainData = trainMax
    testData = testMax
    valData = valMax

    trainData, testData, valData = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)
    
    classTrain = Transformer.toBinaryClassification(activitiesTrain)
    classVal = Transformer.toBinaryClassification(activitiesValidate)
    classTest = Transformer.toBinaryClassification(activitiesTest)
    
    return trainData, testData, valData, classTrain, classTest, classVal, dataLabels


def parameter_rank(top_x, epochs):
    
    trainData, testData, valData, classTrain, classTest, classVal, dataLabels = get_data()
    col_idx = np.arange(0,210)
    probs = []
    for i in range(210):
        probs.append(random.randrange(101))
    probs = np.array(probs)
    for x in range(epochs):
        train_data_df = pd.DataFrame(trainData, columns = dataLabels)
        train_temp_df = train_data_df.copy()
        val_data_df = pd.DataFrame(valData, columns = dataLabels)
        val_temp_df = val_data_df.copy()
        #g = np.array([1])
        #print(probs)
        values = probs
        values = (values - values.min()) / (values - values.min()).sum()
        #display(values, sum(values))
        test_cols = np.random.choice(col_idx, top_x, replace=False, p = values)
        # using set() to perform task
        test_cols_set = set(test_cols)
        col_idx_set = set(col_idx)
        zero_cols = list(col_idx_set - test_cols_set)
        
        train_temp_df.iloc[:, zero_cols] *= 0
        val_temp_df.iloc[:, zero_cols] *= 0
        
        
        Xtrain = train_temp_df.values
        Xval = val_temp_df.values
        accuracy = runBasicNN(Xtrain,classTrain, Xval, classVal)
        curr_val_acc = accuracy[1]
        
        if x >= 1:            
            if curr_val_acc > prev_val_acc:
                np.add.at(probs,test_cols,5)
            elif curr_val_acc > prev_val_acc:
                np.subtract.at(probs,test_cols,5)
            prev_val_acc = accuracy[1]
        else:
            prev_val_acc = accuracy[1]
        
        print(((x+1)/epochs)*100, "% complete")
    
# sorted_index_array = np.argsort(probs)
# # sorted array
# sorted_array = arr[sorted_index_array]
# rslt = sorted_array[-top_x : ]
# top_idx = [i for i, value in enumerate(probs) if value.isin()]
#print(sorted_df)
    result = [dataLabels, probs]
    rslt_df = pd.DataFrame(data = result, columns=['Feature','Probability Weight'])
    sorted_rslt_df = rslt_df.sort_values(by='Probability Weight', ascending=False)

    return sorted_rslt_df
#for x in rslt
        
        

In [243]:
final_df = parameter_rank(20, 10)
display(final_df)


KeyboardInterrupt



In [104]:
compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

labelsMean, trainMean = Transformer.useAverageFD(labelsTrain, compoundDataTrain)
_, testMean = Transformer.useAverageFD(labelsTest, compoundDataTest)
_, valMean = Transformer.useAverageFD(labelsValidate, compoundDataValidate)

labelsMax, trainMax = Transformer.useMaxFD(labelsTrain, compoundDataTrain)
_, testMax = Transformer.useMaxFD(labelsTest, compoundDataTest)
_, valMax = Transformer.useMaxFD(labelsValidate, compoundDataValidate)

#after transformations are done assign data
dataLabels = labelsMax
trainData = trainMax
testData = testMax
valData = valMax

trainData, testData, valData = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)

print(np.shape(trainData))
# for i in range(len(dataLabels)):
#     print(i, "\b:\t", dataLabels[i], "\t", trainData[0:3,i])

(2717, 210)


In [204]:

df = pd.DataFrame(trainData, columns = dataLabels)
#df['fusion_score_max'].values[:] = 0
new_df = df.copy()
g = np.array([1])
new_df.iloc[:, g] *= 0
display(new_df,new_df.values.shape)
# display(df)
# col_idx = np.arange(0,210).tolist()
# probs = []
# for i in range(210):
#     probs.append(random.randrange(100))

# set_col = set(col_idx) 
# test_set = random.choices(list(set_col), weights=probs, k = 190)
# #np.random.choice(col_idx, 20, p = np.divide(probs, 210))
# print(probs, '\n', len(probs), '\n',  test_set, len(np.unique(test_set)))
#     #print(y)
# # #print(lst) 
# # #df.iloc[:, [1, 3]]
#runBasicNN(Xtrain, Ytrain, labelsTrain, compoundDataTest, compoundDataValidate,classVal)

,docking_score_max,fusion_score_max,maxestateindex,minestateindex,maxabsestateindex,minabsestateindex,qed,molwt,heavyatommolwt,exactmolwt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,-0.757445,-0.0,-0.532299,0.783748,-0.532299,-0.478078,0.613944,-0.049216,-0.036600,-0.054705,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
1,-0.324506,0.0,0.046013,0.454683,0.046013,-0.134985,-0.121701,-0.090160,-0.098804,-0.087763,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
2,1.299018,0.0,0.817350,-1.087599,0.817350,-0.552058,0.147804,0.430131,0.495777,0.427890,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
3,0.433139,0.0,-0.135660,0.636055,-0.135660,0.193982,-0.766430,-0.072747,-0.025248,-0.071805,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,6.195526,0.0,4.939209,-0.147946,-0.225465
4,1.515488,0.0,0.375931,-1.092691,0.375931,-0.282160,0.129007,0.650594,0.713922,0.647771,...,-0.176819,2.215875,5.063678,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,0.216669,0.0,0.637921,0.330586,0.637921,-0.088421,-0.647933,0.273846,0.282432,0.276242,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
2713,-1.839794,-0.0,-0.977075,0.613088,-0.977075,1.121948,1.059066,-1.591644,-1.517007,-1.596949,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
2714,-0.324506,-0.0,0.007642,0.606512,0.007642,0.470325,1.001168,-0.701391,-0.672005,-0.706917,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,-0.190696,-0.147946,-0.225465
2715,-0.108036,-0.0,0.302370,0.861139,0.302370,-0.741856,-0.342031,-0.216619,-0.225280,-0.215000,...,-0.176819,-0.369154,-0.185469,-0.060779,-0.103869,-0.147068,0.0,4.939209,-0.147946,-0.225465


(2717, 210)

In [210]:
col_idx = np.arange(0,210)
probs = []
for i in range(210):
    probs.append(random.randrange(101))
print(probs)
values = np.array(probs)
values = (values - values.min()) / (values - values.min()).sum()
#display(values, sum(values))
test = np.random.choice(col_idx, 20, replace=False, p = values)
display(test, len(np.unique(test)))

[2, 42, 72, 42, 21, 77, 39, 42, 64, 32, 36, 60, 28, 91, 48, 96, 78, 23, 27, 83, 36, 53, 37, 97, 69, 54, 61, 62, 60, 80, 48, 51, 93, 11, 23, 13, 33, 15, 6, 95, 26, 87, 77, 40, 39, 67, 21, 79, 31, 21, 39, 57, 13, 31, 96, 90, 45, 23, 21, 94, 17, 3, 100, 30, 21, 50, 31, 94, 55, 98, 0, 38, 98, 94, 25, 83, 63, 37, 37, 58, 81, 0, 51, 81, 78, 33, 84, 100, 7, 12, 96, 8, 16, 57, 40, 51, 44, 87, 43, 20, 78, 93, 2, 68, 18, 2, 81, 94, 93, 82, 44, 28, 17, 14, 87, 8, 72, 18, 21, 9, 6, 69, 62, 48, 100, 60, 29, 62, 90, 6, 81, 40, 1, 32, 73, 28, 29, 76, 72, 14, 52, 74, 31, 59, 77, 0, 86, 61, 51, 53, 28, 37, 48, 14, 37, 29, 56, 94, 17, 11, 61, 58, 88, 27, 11, 94, 0, 20, 72, 9, 65, 25, 38, 30, 28, 60, 1, 9, 44, 17, 12, 43, 29, 89, 63, 28, 96, 19, 75, 26, 14, 91, 6, 29, 65, 87, 71, 4, 57, 16, 40, 78, 94, 79, 87, 43, 20, 88, 91, 85]


array([ 67,  18, 107,   6, 188, 140, 155, 194, 150,  66, 130, 123, 138,
       209, 144,  13, 125, 141, 175,  45])

20

In [234]:

for x in range(3):
    print(x)

g = set(np.array([1,2,3]))
y = set(np.array([2]))
t = list(g-y)
print(t)
    
a = np.array([1, 1, 1, 1, 1])
b = np.array([1,4])
# c = [1,4]
# np.add.at(a, b, 5)
# a
top_idx = np.isin(b,a)
print(top_idx)
idx = [i for i, value in enumerate(top_idx) if value == True]
print(idx)

0
1
2
[1, 3]
[ True False]
[0]
